<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/deepseek7b_quantisierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Pakete ---
%%writefile requirements.txt
transformers
accelerate
bitsandbytes
datasets
evaluate
sacrebleu
codecarbon>=2.5,<3.0
pynvml>=11.5.0
psutil
numpy
pandas
huggingface_hub

Writing requirements.txt


In [ ]:
!pip install -q -r requirements.txt
!pip uninstall -y -q google-genai firebase-admin || true

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.38.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is inco

In [ ]:
# --- Drive mount ---
from google.colab import drive
drive.mount('/content/drive')

# --- Projektordner setzen (muss existieren) ---
import os, pathlib, re, sys
project_path = "/content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_quantisierung"
if not os.path.isdir(project_path):
    raise FileNotFoundError(f"Ordner nicht gefunden: {project_path} bitte manuell anlegen.")
os.chdir(project_path)
print("Arbeitsordner:", os.getcwd())

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/deepseek_quantisierung


In [ ]:
# --- HF Login via Colab-Secret ---
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
if hf_token: login(hf_token); print("Hugging Face Login erfolgreich!")
else:        print("WARNUNG: Kein HF_TOKEN gefunden.")

Hugging Face Login erfolgreich!


In [ ]:
# --- Imports & Setup ---
import warnings; warnings.filterwarnings("ignore")
import time, math, gc, platform, inspect
from dataclasses import dataclass, asdict
from contextlib import nullcontext
from typing import Optional, Tuple
from types import SimpleNamespace

import psutil, numpy as np, pandas as pd, torch
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed
from codecarbon import EmissionsTracker

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    vram_total_gb = torch.cuda.get_device_properties(0).total_memory/(1024**3)
    torch.backends.cuda.matmul.allow_tf32 = True   # T4 nutzt kein TF32, harmless
else:
    gpu_name, vram_total_gb = "CPU", 0.0
print(f"Device: {device} | GPU: {gpu_name} | VRAM: {vram_total_gb:.1f} GB | Torch {torch.__version__}")

Device: cuda | GPU: NVIDIA A100-SXM4-40GB | VRAM: 39.6 GB | Torch 2.8.0+cu126


In [ ]:
# --- Standort / CodeCarbon (Berlin via GCP europe-west10) ---
USE_GCP_REGION = True
COUNTRY_ISO_CODE = "DEU"
CLOUD_PROVIDER   = "google"
CLOUD_REGION     = "europe-west10"
import os as _os, time as _time
_os.environ["CODECARBON_ALLOW_MULTIPLE_RUNS"] = "true"  # unterbindet Lock-Konflikte

def _tracker_base():
    base = dict(log_level="error", output_dir=".")
    sig  = inspect.signature(EmissionsTracker.__init__)
    if "measure_power_secs" in sig.parameters: base["measure_power_secs"] = 1
    if "tracking_mode"      in sig.parameters: base["tracking_mode"]      = "process"
    if USE_GCP_REGION:
        if "cloud_provider" in sig.parameters: base["cloud_provider"] = CLOUD_PROVIDER
        if "cloud_region"   in sig.parameters: base["cloud_region"]   = CLOUD_REGION
        if "country_iso_code" in sig.parameters: base["country_iso_code"] = COUNTRY_ISO_CODE
    else:
        if "country_iso_code" in sig.parameters: base["country_iso_code"] = COUNTRY_ISO_CODE
    return base

def _make_tracker(name, out_csv):
    # frischer Cache + Lock cleanup
    cache_dir = f"/content/.codecarbon_cache_{name}_{int(_time.time())}"
    _os.environ["CODECARBON_CACHE_DIR"] = cache_dir
    for d in (_os.path.expanduser("~/.codecarbon"), "/content/.codecarbon"):
        lf = _os.path.join(d, "codecarbon.lock")
        if os.path.exists(lf):
            try: os.remove(lf)
            except: pass
    return EmissionsTracker(project_name=name, output_file=out_csv, **_tracker_base())

def _safe_start(tr):
    try: tr.start(); return True
    except Exception as e: print("[CodeCarbon] Start-Fehler:", e); return False

def _safe_stop(tr, started):
    if not started: return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try: return tr.stop()
    except Exception as e: print("[CodeCarbon] Stop-Fehler:", e); return SimpleNamespace(energy_consumed=0.0, emissions=0.0)

def _unpack(e):
    if e is None: return 0.0, 0.0
    if hasattr(e,"energy_consumed") and hasattr(e,"emissions"):
        return float(e.energy_consumed), float(e.emissions)
    if isinstance(e, dict):
        return float(e.get("energy_consumed",0.0)), float(e.get("emissions", e.get("emissions_kg",0.0)))
    try: return 0.0, float(e)
    except: return 0.0, 0.0

def _read_energy_csv(path):
    try:
        if not os.path.exists(path): return 0.0
        df = pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            n=c.lower()
            if "energy" in n and "kwh" in n: return float(df[c].iloc[-1])
    except: pass
    return 0.0

def measure(phase, fn, prefix):
    log = f"{prefix}_{phase}.csv"
    tr  = _make_tracker(f"{prefix}_{phase}", log)
    import time as _t
    st=_safe_start(tr); t0=_t.time(); res=fn(); t1=_t.time()
    em =_safe_stop(tr,st); ekwh,co2=_unpack(em)
    if ekwh==0.0:
        ekwh=_read_energy_csv(log)
    return {"phase":phase,"time_s":t1-t0,"energy_kwh":ekwh,"co2_kg":co2}, res

# --- Eval-Konfiguration & Helfer ---
TEACHER_ID = "deepseek-ai/deepseek-llm-7b-base"
PROMPTS = [
  "Explain in simple terms what quantization in neural networks does.",
  "Name three trade-offs when compressing large language models.",
  "Write a short paragraph about energy-efficient AI."
]
EVAL = {
  "max_new_tokens": 32,
  "ppl":  {"name":"wikitext","config":"wikitext-2-raw-v1","split":"test[:1%]"},
  "bleu": {"name":"wmt14","config":"de-en","split":"test[:32]"},
}
def parse_subset_count(split, default=32):
    m = re.search(r":\s*(\d+)\s*\]$", split or "")
    return int(m.group(1)) if m else default
BLEU_N = parse_subset_count(EVAL["bleu"]["split"], 32)

def autocast_ctx():
    return torch.autocast(device_type="cuda", dtype=torch.float16) if device=="cuda" else nullcontext()

def safe_max_len(tok, model, fallback=2048, upper=100000):
    cand=getattr(tok,"model_max_length",None)
    if isinstance(cand,int) and 0<cand<upper: return cand
    cand=getattr(getattr(model,"config",None),"max_position_embeddings",None)
    if isinstance(cand,int) and 0<cand<upper: return cand
    return fallback

def simple_generate(model, tok, prompts, max_new_tokens=32):
    model.eval(); max_len=safe_max_len(tok, model); texts=[]; total_tokens=0
    with torch.no_grad(), autocast_ctx():
        for p in prompts:
            enc = tok(p, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
            room = max_len - enc["input_ids"].shape[1]
            cur  = max(1, min(max_new_tokens, int(room)))
            out  = model.generate(**enc, max_new_tokens=cur, do_sample=False, pad_token_id=tok.eos_token_id)
            total_tokens += int(out.shape[1]-enc["input_ids"].shape[1])
            texts.append(tok.decode(out[0], skip_special_tokens=True))
    return texts, total_tokens

bleu_metric = evaluate.load("sacrebleu")
def eval_bleu(model, tok, cfg, max_new_tokens=32):
    ds = load_dataset(cfg["name"], cfg["config"], split=cfg["split"])
    max_len=safe_max_len(tok, model); preds, refs = [], []
    with torch.no_grad(), autocast_ctx():
        for ex in ds:
            de, en = ex["translation"]["de"], ex["translation"]["en"]
            prompt = f"Translate to English:\nGerman: {de}\nEnglish:"
            enc = tok(prompt, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
            room = max_len - enc["input_ids"].shape[1]
            cur  = max(1, min(max_new_tokens, int(room)))
            out  = model.generate(**enc, max_new_tokens=cur, do_sample=False, pad_token_id=tok.eos_token_id)
            gen  = tok.decode(out[0], skip_special_tokens=True)
            hyp  = gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
            preds.append(hyp); refs.append([en])
    return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

def eval_ppl(model, tok, cfg):
    ds = load_dataset(cfg["name"], cfg["config"], split=cfg["split"])
    max_len=safe_max_len(tok, model); losses=[]
    with torch.no_grad():
        for t in ds["text"]:
            if not isinstance(t,str) or len(t.strip())<4: continue
            enc = tok(t, return_tensors="pt", truncation=True, max_length=max_len)
            ids = enc["input_ids"].to(model.device)
            with autocast_ctx(): out = model(ids, labels=ids)
            losses.append(float(out.loss.detach().cpu()))
    return math.exp(np.mean(losses)) if losses else None

# --- Loader je Quantisierungs-Variante ---
def load_model_variant(variant:str):
    tok = AutoTokenizer.from_pretrained(TEACHER_ID, use_fast=True)
    tok.pad_token = tok.eos_token
    if variant=="8bit":
        bnb = BitsAndBytesConfig(load_in_8bit=True)
        model = AutoModelForCausalLM.from_pretrained(
            TEACHER_ID, device_map="auto", quantization_config=bnb, attn_implementation="sdpa"
        )
        prec = "int8"
    elif variant=="4bit":
        bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                                 bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.float16)
        model = AutoModelForCausalLM.from_pretrained(
            TEACHER_ID, device_map="auto", quantization_config=bnb, attn_implementation="sdpa"
        )
        prec = "int4-nf4"
    else:
        raise ValueError("variant must be '8bit' or '4bit'")
    model.eval()
    return tok, model, prec

In [ ]:
# --- Run & speichern ---
def run_variant(variant:str, alias="ds7b"):
    print(f"\n### DeepSeek-7B – {variant}")
    prefix = f"deepseek_quantisierung_{variant}_{alias}"

    tok, model, prec = load_model_variant(variant)

    m_gen,  (samples, n_tok) = measure("gen",  lambda: simple_generate(model, tok, PROMPTS, EVAL["max_new_tokens"]), prefix)
    m_ppl,  ppl              = measure("ppl",  lambda: eval_ppl(model, tok, EVAL["ppl"]),  prefix)
    if device=="cuda": torch.cuda.empty_cache()
    m_bleu, bleu            = measure("bleu", lambda: eval_bleu(model, tok, EVAL["bleu"], EVAL["max_new_tokens"]), prefix)

    ram = psutil.Process().memory_info().rss/(1024**3)
    valloc = torch.cuda.memory_allocated()/(1024**3) if device=="cuda" else 0.0
    vres  = torch.cuda.memory_reserved() /(1024**3) if device=="cuda" else 0.0

    row = dict(model_id=TEACHER_ID, alias=alias, variant=variant, precision=prec,
               time_s=m_gen["time_s"]+m_ppl["time_s"]+m_bleu["time_s"],
               energy_kwh=m_gen["energy_kwh"]+m_ppl["energy_kwh"]+m_bleu["energy_kwh"],
               co2_kg=m_gen["co2_kg"]+m_ppl["co2_kg"]+m_bleu["co2_kg"],
               tokens_out=int(n_tok), ppl=ppl, bleu=bleu,
               ram_GB=ram, vram_alloc_GB=valloc, vram_reserved_GB=vres,
               notes=f"GPU={gpu_name}, VRAM={vram_total_gb:.1f} GB")
    phases = []
    for m in [m_gen,m_ppl,m_bleu]:
        phases.append(dict(phase=m["phase"], time_s=m["time_s"], energy_kwh=m["energy_kwh"], co2_kg=m["co2_kg"], alias=alias, variant=variant))
    # Samples sichern
    with open(f"deepseek_quantisierung_samples_{variant}.txt","w",encoding="utf-8") as f:
        for i,txt in enumerate(samples,1): f.write(f"--- Beispiel {i} ({variant}) ---\n{txt}\n\n")
    return row, phases

all_rows, all_phases = [], []
for v in ["8bit","4bit"]:
    row, phases = run_variant(v)
    all_rows.append(row); all_phases.extend(phases)

df  = pd.DataFrame(all_rows)
df["kg_per_kwh"] = (df["co2_kg"]/df["energy_kwh"]).replace([np.inf,-np.inf], np.nan)
dfp = pd.DataFrame(all_phases)

df.to_csv("deepseek_quantisierung_results.csv", index=False)
dfp.to_csv("deepseek_quantisierung_per_phase.csv", index=False)
print("\nGespeichert:")
print(" - deepseek_quantisierung_results.csv")
print(" - deepseek_quantisierung_per_phase.csv")
print("Emission-Logs je Variante: deepseek_quantisierung_{8bit|4bit}_{gen|ppl|bleu}.csv")


### DeepSeek-7B – 8bit


tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

...

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]


### DeepSeek-7B – 4bit


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Gespeichert:
 - deepseek_quantisierung_results.csv
 - deepseek_quantisierung_per_phase.csv
Emission-Logs je Variante: deepseek_quantisierung_{8bit|4bit}_{gen|ppl|bleu}.csv
